This notebook illustrates:  
(1) Various feature selection techniques and evaluation schemes on the movie review dataset;  
(2) Model comparison using cross-validation and paired t-test.

# 1. Feature selection 

In [1]:
import pandas as pd
import numpy as np

#### LOAD DATASETS ####

train_data_file = "movie_train.csv"
test_data_file = "movie_test.csv"

# Import train and test dataset into data frames and print out the original lengths
train_data_df = pd.read_csv(train_data_file)
test_data_df = pd.read_csv(test_data_file)
print ("Original train set: ",len(train_data_df))
print ("Original test set: ",len(test_data_df))

### CLEAN DATASETS ###
# Remove empty rows from both sets and print out the new lengths
train_data_df = train_data_df[~train_data_df["review"].isnull()]
test_data_df = test_data_df[~test_data_df["review"].isnull()]
print ("After removing empty reviews, train set size: ",len(train_data_df))
print ("After removing empty reviews, test set size: ",len(test_data_df))

# Remove rows with null labels
train_data_df = train_data_df[~train_data_df["sentiment"].isnull()]
test_data_df = test_data_df[~test_data_df["sentiment"].isnull()]
print ("After removing instances with no labels, train set size: ", len(train_data_df))
print ("After removing instances with no labels, test set size: ", len(test_data_df))

# print out top 5 rows of the train set
display(train_data_df.head(5))

Original train set:  10000
Original test set:  2500
After removing empty reviews, train set size:  10000
After removing empty reviews, test set size:  2500
After removing instances with no labels, train set size:  10000
After removing instances with no labels, test set size:  2500


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [2]:
# use original reviews for model building
y_train = train_data_df["sentiment"]
y_test = test_data_df["sentiment"]

train_text = train_data_df["review"]
test_text = test_data_df["review"]

### Count-based feature extraction

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,1))

# create training data representation
train_data_cv = vectorizer.fit_transform(train_text)

# observe the words in the created dictionary across the document
print(len(vectorizer.vocabulary_), " ... ", list(vectorizer.vocabulary_.items())[0:100],"\n")

print(train_data_cv.shape,"\n") 

# create test data representation
test_data_cv = vectorizer.transform(test_text)
print(test_data_cv.shape,"\n") 

52522  ...  [('one', 32799), ('of', 32624), ('the', 46654), ('other', 33143), ('reviewers', 38996), ('has', 21177), ('mentioned', 29519), ('that', 46645), ('after', 1404), ('watching', 50735), ('just', 25230), ('oz', 33501), ('episode', 15679), ('you', 52219), ('ll', 27402), ('be', 4335), ('hooked', 22227), ('they', 46753), ('are', 2793), ('right', 39215), ('as', 3012), ('this', 46806), ('is', 24390), ('exactly', 16108), ('what', 51062), ('happened', 21019), ('with', 51540), ('me', 29232), ('br', 6007), ('first', 17476), ('thing', 46779), ('struck', 44844), ('about', 788), ('was', 50689), ('its', 24476), ('brutality', 6474), ('and', 2204), ('unflinching', 48907), ('scenes', 40654), ('violence', 50167), ('which', 51117), ('set', 41482), ('in', 23209), ('from', 18505), ('word', 51720), ('go', 19676), ('trust', 48109), ('not', 32203), ('show', 42050), ('for', 17988), ('faint', 16652), ('hearted', 21401), ('or', 32960), ('timid', 47073), ('pulls', 36772), ('no', 32020), ('punches', 36802),

What is the number of features? Note that it is quite large.

In [4]:
# import relevant libraries
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, auc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

### Train and evaluate the Naive Bayes model

In [5]:
naive_bayes = MultinomialNB()
naive_bayes.fit(train_data_cv, y_train)
predictions = naive_bayes.predict(test_data_cv)

# average could be of 3 kinds = weighted, macro, micro
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="weighted"))
print("Recall score: ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score: ", f1_score(y_test, predictions, average = "weighted"))

Accuracy score:  0.84
Precision score:  0.8414115204180682
Recall score:  0.84
F1 score:  0.8397981792031259


### Low-variance feature removal

In [6]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold = 0.001)

X_train_features_filtered_var_thr = selector.fit(train_data_cv).transform(train_data_cv)
print ("Train feature space before filtering: ", train_data_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr.shape)

X_test_features_filtered_var_thr = selector.transform(test_data_cv)
print ("Test feature space before filtering: ", test_data_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr.shape)

Train feature space before filtering:  (10000, 52522)
Train feature space after filtering:  (10000, 13136)
Test feature space before filtering:  (2500, 52522)
Test feature space after filtering:  (2500, 13136)


In [7]:
X_train_features_filtered_var_thr

<10000x13136 sparse matrix of type '<class 'numpy.int64'>'
	with 1277718 stored elements in Compressed Sparse Row format>

In [8]:
from scipy.sparse import csr_matrix, csr_array

In [9]:
X_train_features_filtered_var_thr.tocoo().row[0:1000]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [10]:
X_train_features_filtered_var_thr.tocoo().row

array([   0,    0,    0, ..., 9999, 9999, 9999], dtype=int32)

What happened with low-variance feature removal? What does the results tell you?

Ans) - Inverse relation, lower thresholds so more features are available

Experiment with different variance threshold values and observe how the number of features changes.

In [11]:
selector = VarianceThreshold(threshold = 0.001)

X_train_features_filtered_var_thr = selector.fit(train_data_cv).transform(train_data_cv)
print ("Train feature space before filtering: ", train_data_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr.shape)

X_test_features_filtered_var_thr = selector.transform(test_data_cv)
print ("Test feature space before filtering: ", test_data_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr.shape)

print("\n")
selector = VarianceThreshold(threshold = 0.005)

X_train_features_filtered_var_thr = selector.fit(train_data_cv).transform(train_data_cv)
print ("Train feature space before filtering: ", train_data_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr.shape)

X_test_features_filtered_var_thr = selector.transform(test_data_cv)
print ("Test feature space before filtering: ", test_data_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr.shape)

Train feature space before filtering:  (10000, 52522)
Train feature space after filtering:  (10000, 13136)
Test feature space before filtering:  (2500, 52522)
Test feature space after filtering:  (2500, 13136)


Train feature space before filtering:  (10000, 52522)
Train feature space after filtering:  (10000, 4901)
Test feature space before filtering:  (2500, 52522)
Test feature space after filtering:  (2500, 4901)


### Feature selection using chi-squared statistic and k-best features

In [12]:
from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=200)
X_train_features_filtered_kbest = selector.fit_transform(train_data_cv, y_train)
print ("Train feature space before filtering: ", train_data_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)

X_test_features_filtered_kbest = selector.transform(test_data_cv)
print ("Test feature space before filtering: ", test_data_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (10000, 52522)
Train feature space after filtering:  (10000, 200)
Test feature space before filtering:  (2500, 52522)
Test feature space after filtering:  (2500, 200)


### Training and evaluating a model with selected features

Use the code from above to train and evaluate a model. Only this time with the `X_train_features_filtered_var_thr` and  `X_test_features_filtered_var_thr` (features selected after low-variance feature removal). Compare the results with the earlier model performance.

In [13]:
# your training/evaluation code here
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_features_filtered_var_thr, y_train)
predictions = naive_bayes.predict(X_test_features_filtered_var_thr)

# average could be of 3 kinds = weighted, macro, micro
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="weighted"))
print("Recall score: ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score: ", f1_score(y_test, predictions, average = "weighted"))

Accuracy score:  0.8224
Precision score:  0.8229600444963653
Recall score:  0.8224
F1 score:  0.8222955791933102


You can train/evaluate the model with features selected using chi-squared statistic, as well.

In [14]:
# your training/evaluation code here
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_features_filtered_kbest, y_train)
predictions = naive_bayes.predict(X_test_features_filtered_kbest)

# average could be of 3 kinds = weighted, macro, micro
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="weighted"))
print("Recall score: ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score: ", f1_score(y_test, predictions, average = "weighted"))

Accuracy score:  0.8252
Precision score:  0.8252047171357175
Recall score:  0.8252
F1 score:  0.8251958884138865


### Model-based feature selection

Here we select features for a logistic regression model with L1 (LASSO) regularization, which implicitly performs feature selection by setting feature weights to zero. See the below code and interpret the printed outputs in the cell. The selector is using an estimator to keep the best.

In [15]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

selector = SelectFromModel(estimator=LogisticRegression(solver="liblinear", penalty="l1")).fit(train_data_cv, y_train)

# estimator_: The base estimator from which the transformer is built. This attribute exists only when fit has been called.
# In this case, the estimator is Logistic Regression.
# coef_ : an attribute of the Logistic Regression estimator. Tha value represents the weight of each of the features fed to the model.
print("Coefficients of the features fed to the ",selector.estimator_ , " estimator")
print(selector.estimator_.coef_)

print()
# threshold_: Threshold value used for feature selection.
print("selector.threshold_ ",selector.threshold_)

print()
# gives the features which were kept or dropped according to the model.
print("Selector support: ", selector.get_support())

Coefficients of the features fed to the  LogisticRegression(penalty='l1', solver='liblinear')  estimator
[[ 0.         -0.05714592  0.         ...  0.          0.
   0.        ]]

selector.threshold_  1e-05

Selector support:  [False  True False ... False False False]


In [16]:
train_data_cv_selectedfrommodel = selector.transform(train_data_cv)
test_data_cv_selectedfrommodel = selector.transform(test_data_cv)

print ("Train feature space before filtering: ", train_data_cv.shape)
print ("Train feature space after filtering: ", train_data_cv_selectedfrommodel.shape)
print ("Test feature space before filtering: ", test_data_cv.shape)
print ("Test feature space after filtering: ", test_data_cv_selectedfrommodel.shape)

Train feature space before filtering:  (10000, 52522)
Train feature space after filtering:  (10000, 2257)
Test feature space before filtering:  (2500, 52522)
Test feature space after filtering:  (2500, 2257)


### Creating pipelines in scikit-learn

scikit-learn pipelines can be used to specify multiple steps in a data modeling process and execute them in sequence. The pipeline below does feature selection, then builds a multinomial NB model using the selected features. 

It is possible to add more steps to the pipeline such as StandardScaler. You can read more about the scikit-learn Pipeline here- https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# define x_all and y_all to complete this code and run to see to results

frames = [train_data_df, test_data_df]

all_dataset = pd.concat(frames)

x_all_text = all_dataset["review"]
y_all = all_dataset["sentiment"]
y_all = np.array(y_all)

x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size = 0.25)
 
pipeline = Pipeline([('var_th', VarianceThreshold(threshold = 0.001)),('mnb', MultinomialNB())], verbose = True)
 
pipeline.fit(x_train, y_train)

# to see all the hyper parameters
print()
print(pipeline.get_params())

print()
print("Evaluation accuracy: ",pipeline.score(x_test, y_test))

NameError: name 'x_all' is not defined

### Stratified k-fold cross-validation

Below is an example of how the stratified k-fold cross-validation divides the data into training and test sets based on the value of number of splits. You can update this code further to incorporate the model training and evaluation for each split and compare the results.

In [ ]:
import numpy as np
## combined dataset
frames = [train_data_df, test_data_df]

all_dataset = pd.concat(frames)

x_all_text = all_dataset["review"]
y_all = all_dataset["sentiment"]
y_all = np.array(y_all)

vectorizer = CountVectorizer(ngram_range = (1,1))
x_all = vectorizer.fit_transform(x_all_text)

print(len(vectorizer.vocabulary_), " ... ", list(vectorizer.vocabulary_.items())[0:100],"\n")
print(x_all.shape,"\n") 
print(y_all.shape,"\n") 

# create test data representation
# test_data_cv = vectorizer.transform(test_text)
# print(test_data_cv.shape,"\n") 

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np
from sklearn.pipeline import Pipeline

skf = StratifiedKFold(n_splits=5)

print(skf)

for train_index, test_index in skf.split(x_all, y_all):
    print("TRAIN:", type(train_index))
    print("TEST:", test_index)
    print()
    X_train = x_all[train_index]
    X_test = x_all[test_index]
    Y_train = y_all[train_index]
    Y_test = y_all[test_index]

    pipeline = Pipeline([('chi2', SelectKBest(chi2, k=200)),('mnb', MultinomialNB())], verbose = True)

    pipeline.fit(X_train, Y_train)

    # to see all the hyper parameters
    print()
    print(pipeline.get_params())

    print()
    print("Evaluation accuracy: ",pipeline.score(X_test, Y_test))
    print("\n\n.........................\n\n")

Below is a one liner code to implement an ML pipeline with Sklearn Pipeline and cross validation

In [ ]:
pipeline = Pipeline([('chi2', SelectKBest(chi2, k=200)),('mnb', MultinomialNB())], verbose = True)
scores = cross_val_score(pipeline, x_all, y_all, cv=5)
print("Scores of the K fold stratified cross validations= ", scores)
print("Average score of the K fold stratified cross validation= ", scores.mean())

# 2. Model comparison

### Count-based feature extraction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Combine the training and test data because we'd like to do cross-validation
frames = [train_data_df, test_data_df]

all_dataset = pd.concat(frames)

x_all_text = all_dataset["review"]
y_all = all_dataset["sentiment"]
y_all = np.array(y_all)

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,1))

# create training data representation
x_all = vectorizer.fit_transform(x_all_text)

# observe the words in the created dictionary across the document
print(len(vectorizer.vocabulary_), " ... ", list(vectorizer.vocabulary_.items())[0:100],"\n")

print(x_all.shape,"\n") 

### Model comparison across k-fold cross-validation

We train 2 different models (Multinomial Naive Bayes and Logistic Regression) on the complete dataset. We use stratified K-fold to split data into train and test sets with roughly similar proportions of each label. In each fold, we compare the performance of the two models on the same test split.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
#from sklearn.model_selection import cross_validate
import math
from scipy import stats

from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

skf = StratifiedKFold(n_splits=5)

print(skf)


i=0
f1s_nb = []
f1s_lr = []

for train_index, test_index in skf.split(x_all, y_all):
    print("Training instance indexes:", train_index)
    print("Test instance indexes:", test_index)
    X_train = x_all[train_index]
    X_test = x_all[test_index]
    Y_train = y_all[train_index]
    Y_test = y_all[test_index]

    naive_bayes = MultinomialNB()
    naive_bayes.fit(X_train, Y_train)
    predictions_nb = naive_bayes.predict(X_test)

    f1_nb = f1_score(Y_test, predictions_nb, average = "weighted")
    f1s_nb.append(f1_nb)
    
    LR = LogisticRegression(solver="liblinear", penalty="l1")
    LR.fit(X_train, Y_train)
    predictions_lr = LR.predict(X_test)

    f1_lr = f1_score(Y_test, predictions_lr, average = "weighted")
    f1s_lr.append(f1_lr)
    
    print("F1 scores at fold ", i+1  ," for NB and LR are : ", f1_nb , f1_lr)
    print()
    i+=1

### Measuring statistical significance of the model performance using Student's t-test

Because we have results from 5-fold cross validation, we can measure if there's a statistically significant difference between the mean F1 scores of the two classifiers.

In [ ]:
from scipy import stats

# Student's t-test 
nb_lr_ttest = stats.ttest_ind(f1s_nb,f1s_lr)
print("t-test result: ", nb_lr_ttest) 

What does the result above indicate, if the significance level is 95%? How about 99.9%? 

We can calculate confidence intervals for accuracy (since we can treat the classification outcome as "accurate" or "not accurate") of Naive Bayes and logistic regression classifiers. You can learn more about how to calculate confidence intervals for accuracy here: https://machinelearningmastery.com/confidence-intervals-for-machine-learning/.

### Ablation test

To test how important individual features (or components of a model) are, we can conduct an ablation test.
- Train the full model with all features included and conduct evaluation
- Remove feature (or group of features) and conduct evaluation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# use the original training/test split
x_train = train_data_df["review"]
x_test = test_data_df["review"]
y_train = train_data_df["sentiment"]
y_test = test_data_df["sentiment"]

# we will test different n-gram features for their contribution to logistic regression classifier
def ablation_test(x_train, y_train, x_test, y_test, ngram_range):
# set the n-gram range

    vectorizer = CountVectorizer(ngram_range = ngram_range)

    # create training data representation
    grams = vectorizer.fit(x_train)
    
    x_train = vectorizer.transform(x_train)
    x_test = vectorizer.transform(x_test)
    
    # observe the words in the created dictionary across the document
    print("number of features for ngram_range ", ngram_range, " : ",len(vectorizer.vocabulary_))

    LR = LogisticRegression(solver="liblinear", penalty="l1")
    LR.fit(x_train, y_train)
    predictions_lr = LR.predict(x_test)

    f1_lr = f1_score(y_test, predictions_lr, average = "weighted")
    print("F1 score for ", ngram_range, " : ",f1_lr, "\n")

In [ ]:
# run ablation test for unigrams, unigrams+bigrams, unigrams+bigrams+trigrams, etc.
for i in range(1,4):
    for j in range(3,0,-1): 
        if(i<=j):
            ablation_test(x_train, y_train, x_test, y_test, (i,j))

What do the results indicate? Which n-gram feature is most useful? least useful? Do the results align with your expectations?